In [1]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')

In [6]:
train.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [7]:
train.discourse_effectiveness.value_counts()

Adequate       20977
Effective       9326
Ineffective     6462
Name: discourse_effectiveness, dtype: int64

In [9]:
train.groupby('discourse_type')['discourse_effectiveness'].value_counts()

discourse_type        discourse_effectiveness
Claim                 Adequate                   7097
                      Effective                  3405
                      Ineffective                1475
Concluding Statement  Adequate                   1945
                      Effective                   825
                      Ineffective                 581
Counterclaim          Adequate                   1150
                      Effective                   418
                      Ineffective                 205
Evidence              Adequate                   6064
                      Ineffective                3156
                      Effective                  2885
Lead                  Adequate                   1244
                      Effective                   683
                      Ineffective                 364
Position              Adequate                   2784
                      Effective                   770
                      Ineffective   

In [10]:
preds = pd.read_csv('../output/HF-32-OOF.csv')

In [11]:
preds.groupby(['discourse_type', 'discourse_effectiveness'])['loss'].mean()

discourse_type        discourse_effectiveness
Claim                 Adequate                   0.397092
                      Effective                  0.595974
                      Ineffective                1.545117
Concluding Statement  Adequate                   0.424606
                      Effective                  0.487241
                      Ineffective                0.970177
Counterclaim          Adequate                   0.352546
                      Effective                  0.789613
                      Ineffective                1.631169
Evidence              Adequate                   0.533110
                      Effective                  0.531594
                      Ineffective                0.826802
Lead                  Adequate                   0.476868
                      Effective                  0.589001
                      Ineffective                1.200841
Position              Adequate                   0.294640
                      Effe

In [13]:
train.groupby('discourse_effectiveness')['essay_id'].nunique()

discourse_effectiveness
Adequate       3853
Effective      1534
Ineffective    3207
Name: essay_id, dtype: int64

In [14]:
preds.sort_values('loss', ascending=False)

,discourse_id,preds,labels,discourse_type,discourse_effectiveness,discourse_text,loss
18416,5a82a0a5e324,[1.8230322e-01 6.7045586e-04 8.1702632e-01],1,Position,Effective,Venus is a worthy planet because it does not h...,7.307553
10906,749e46ad80ae,[0.03029583 0.96765655 0.00204761],2,Evidence,Ineffective,"Most people are too busy, or lazy, or just don...",6.191081
10910,81a1eb3bf903,[0.03997533 0.95758057 0.00244417],2,Evidence,Ineffective,The majority does not follow the consitiution ...,6.014049
20415,62f8f34f72b2,[0.9342819 0.00282638 0.06289168],1,Claim,Effective,If you dont have one you have to walk which me...,5.868758
24513,4c819b58e66f,[0.8213581 0.00308028 0.17556167],1,Concluding Statement,Effective,This is why and how dangerous Venus can be and...,5.782733
...,...,...,...,...,...,...,...
36763,532bb13396d8,[9.5754704e-03 7.3802052e-04 9.8968655e-01],2,Evidence,Ineffective,Venus is a planet what belong the System Solar...,0.010367
28879,bb16a52c39be,[9.432004e-03 8.412355e-04 9.897267e-01],2,Evidence,Ineffective,In the story The Challenge Of Exploring Venus ...,0.010326
28646,6c7a6e7f52ab,[9.2317788e-03 9.9023741e-01 5.3083437e-04],1,Evidence,Effective,"Such skills include presenting, leading, criti...",0.009811
29322,c054a9fdc7fb,[0.00623465 0.00105832 0.9927071 ],2,Evidence,Ineffective,"In this article i read about ""Making Mona Lisa...",0.007320


In [48]:
train[train.discourse_id == '4c819b58e66f']

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
4415,4c819b58e66f,4EBE610AF54A,This is why and how dangerous Venus can be and...,Concluding Statement,Effective


In [25]:
label2id = {'Adequate': 0, 'Effective': 1, 'Ineffective': 2}
id2label = {v:k for k,v in label2id.items()}
id2label

{0: 'Adequate', 1: 'Effective', 2: 'Ineffective'}

In [35]:
from ast import literal_eval

In [44]:
# 5a82a0a5e324
def print_pred(essay_id):
    for i, row in train[train.essay_id == essay_id].iterrows():
        predsrow = preds[preds.discourse_id == row.discourse_id].reset_index(drop=True).loc[0]
        print(f'{row.discourse_type}: {row.discourse_effectiveness} (loss: {predsrow.loss})')
        print('Adequate | Effective | Ineffective')
        print(predsrow.preds)
        # pred = id2label[np.argmax(literal_eval(predsrow.preds))]
        # print(f'Pred: {pred} (conf: {max(predsrow.preds)}')
        print(row.discourse_text)
        print()
        
print_pred('85A320C7B916')

Lead: Adequate (loss: 1.3530118465423584)
Adequate | Effective | Ineffective
[0.25846064 0.00092671 0.7406126 ]
The author studies Venus the way some people study science and how others study math and English. The auhtor studies really hard on what he wants and needs to know. 

Claim: Adequate (loss: 0.7728095650672913)
Adequate | Effective | Ineffective
[0.46171403 0.00111687 0.5371691 ]
Some dangers that it persents would be that not all of Venus is safe and not all of space is safe either 

Position: Effective (loss: 7.307552814483643)
Adequate | Effective | Ineffective
[1.8230322e-01 6.7045586e-04 8.1702632e-01]
Venus is a worthy planet because it does not have all of man kind on it destroying it or usig it. Venus is a place where some people go to see outisde of our world to see what space really does look like. 

Evidence: Ineffective (loss: 0.1357495933771133)
Adequate | Effective | Ineffective
[0.12583782 0.00110101 0.87306124]
In parargraph 1 it talks about about of Venues is 

In [45]:
print_pred('7263826C2217')

Lead: Effective (loss: 0.1243980601429939)
Adequate | Effective | Ineffective
[0.11030828 0.8830282  0.00666346]
The electoral college has been around for as long as our nation has reined over this land. I fervently believe we should keep the electoral college as it is. We, as people, are not ready for the responability of choosing our next leader directly viva popular vote. Some of us might be ready for this responability now, but the magority of America cannot handel this responibity. The majority has stared riots over lost football games, acknolaged said riots and justifyed them then turn around and see a riot about a kid being shot 6 times by an officer and send in swat teams and start throwing smoke bombs. Because of these actions and the many like it, 

Position: Effective (loss: 0.2602515816688537)
Adequate | Effective | Ineffective
[0.19913599 0.7708576  0.03000638]
I whole-heartedly believe if we leave the responability to chose our president in the magority vote that this cou

In [47]:
print_pred('C74CBEFD0B9E')

Position: Ineffective (loss: 0.8389051556587219)
Adequate | Effective | Ineffective
[0.5642242  0.00359237 0.43218344]
There are many ways to limiting car usage 

Claim: Adequate (loss: 0.0344767868518829)
Adequate | Effective | Ineffective
[0.9661107  0.00739077 0.02649847]
it would be better for the enviorment 

Claim: Adequate (loss: 0.1324410587549209)
Adequate | Effective | Ineffective
[0.8759545  0.00928536 0.11476005]
more cash in ur poket 

Evidence: Adequate (loss: 0.1269784718751907)
Adequate | Effective | Ineffective
[0.88075256 0.00235596 0.11689143]
useing less gas 

Claim: Ineffective (loss: 3.0790610313415527)
Adequate | Effective | Ineffective
[0.9499449  0.00405266 0.04600243]
it would be much safer for your if it gona have a cerfiu. 

Claim: Adequate (loss: 0.1501583755016327)
Adequate | Effective | Ineffective
[0.8605716  0.00112189 0.13830645]
getting to places faster 

Claim: Effective (loss: 5.868758201599121)
Adequate | Effective | Ineffective
[0.9342819  0.00282

In [49]:
print_pred('4EBE610AF54A')

Lead: Adequate (loss: 0.2651020884513855)
Adequate | Effective | Ineffective
[0.76712763 0.00433142 0.22854102]
Would it be safe to travle to Venus or what about to colect some samples of Venus like rocks,gasses or other things in Venus? Have people wounder how is Venus and is it simliller or different to Earth 

Position: Ineffective (loss: 0.3508494198322296)
Adequate | Effective | Ineffective
[0.29236397 0.00354625 0.7040897 ]
We know that Venus is a planet and that Venus is closest to Earth but some poeple might not know how danger Venus has. Has any people or a ship landed on Venus or expernece how Venus feels or looks like on ground. Is it is real worth going to Venus and expercence there conditions or find any life forms in Venus? 

Claim: Adequate (loss: 0.730544924736023)
Adequate | Effective | Ineffective
[0.48164645 0.00162231 0.5167312 ]
Venus Is a similler planet like Earth but only its density and size of the planet. 

Evidence: Adequate (loss: 0.2871854305267334)
Adequat